In [16]:
from bs4 import Tag, NavigableString, BeautifulSoup
from pprint import pprint
import urllib
import json
import pandas

In [17]:
colnames = ['url']
data = pandas.read_csv('list_of_bands.csv', names=colnames)
urls = data.url.tolist()

In [18]:

# probably gonna delete this

# def get_info_from_cell(cell):
#     cell_info = ['']
#     for element in cell.contents:
#         if isinstance(element, NavigableString):
#             cell_info[-1] += str(element)
#         elif element.name == 'br':
#             cell_info.append('')
#         else:
#             cell_info[-1] += str(element.text)        
    
#     return cell_info

In [19]:
def get_info_from_cell(cell):
    cell_info = cell.text.split('\n')
    return [s for s in cell_info if s != '']

In [20]:
def parse_infobox(soup):
    
    infobox_dict = {}

    infobox_rows = soup.find("table", {"class": "infobox"}).tbody.contents
    for row in infobox_rows:
        if len(row.contents) != 2:
            continue
        cell_1 = row.contents[0]
        cell_2 = row.contents[1]
        if cell_1.name != "th" or cell_2.name != "td":
            continue

        infobox_dict[cell_1.text] = get_info_from_cell(cell_2)
    
    return infobox_dict

In [22]:
fileNum = 0
for url in urls:
    html_doc = urllib.request.urlopen(url).read()
    soup = BeautifulSoup(html_doc, 'html.parser')
    with open('./data/info-'+str(fileNum)+'.json', 'w', encoding="utf-8") as outfile:
        try:
            json.dump(parse_infobox(soup), outfile, indent = 3, ensure_ascii = False)
        except:
            continue
        
    fileNum += 1